### Consulta 1
def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.


In [107]:
import pandas as pd

In [108]:
columns = ['id', 'price']
games_price = pd.read_csv(r'./datasets/steam_games.csv', usecols=columns)
games_price.head()

,price,id
0,4.99,761140.0
1,Free To Play,643980.0
2,Free to Play,670290.0
3,0.99,767400.0
4,2.99,773570.0


In [109]:
games_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   30758 non-null  object 
 1   id      32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [123]:
def fix_price(df):

    errors_list = []
    for i in df['price']:
        try:
            float(i)
        except:
            errors_list.append(i)

    errors = set(errors_list)
    #uniques_not_free = ['Starting at $499.00', 'Starting at $449.00']
    df['price_fixed'] = df['price'].apply(lambda x: 0 if x in errors 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
    return df

In [112]:
games_price['price_fixed'] = games_price['price'].apply(lambda x: 0 if x in uniques 
                                                        else 499.0 if x=='Starting at $499.00'
                                                        else 449.0 if x=='Starting at $449.00'
                                                        else x)
games_price.head()

,price,id,price_fixed
0,4.99,761140.0,4.99
1,Free To Play,643980.0,0
2,Free to Play,670290.0,0
3,0.99,767400.0,0.99
4,2.99,773570.0,2.99


In [113]:
games_price['price_fixed'] = games_price['price_fixed'].astype(float)

In [114]:
games_price.rename(columns={'id':'item_id'},inplace=True)
games_price.drop(columns='price', inplace=True)
games_price.head()

,item_id,price_fixed
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


In [115]:
games_price.isna().sum()

item_id           2
price_fixed    1377
dtype: int64

In [116]:
games_price.fillna(0, inplace=True)

In [117]:
games_price.isna().sum()

item_id        0
price_fixed    0
dtype: int64

In [118]:
columns = ['user_id', 'item_id', 'items_count']
user_items = pd.read_csv('datasets/aus_users_items.csv', usecols=columns)
user_items.head()

,user_id,items_count,item_id
0,76561197970982479,277,10
1,76561197970982479,277,20
2,76561197970982479,277,30
3,76561197970982479,277,40
4,76561197970982479,277,50


In [119]:
df_merged = user_items.merge(games_price, on='item_id', how='left')
df_merged.sample(10)

,user_id,items_count,item_id,price_fixed
856758,76561198008592438,294,28050,NaN
2614458,76561198064945689,155,108800,29.99
1430116,Victorypower,217,264240,9.99
5089894,76561198101205256,144,218230,0.00
4554080,76561198058959788,56,340,0.00
2974978,091263,369,70000,9.99
264034,76561198051309911,28,38410,9.99
2892329,nolongerliving,604,22230,9.99
191383,76561198080592688,242,411190,NaN
3343785,76561198079524591,66,268850,0.00


In [120]:
user_spent = df_merged.groupby('user_id')['price_fixed'].agg('sum')
user_spent = pd.DataFrame(user_spent)
user_spent.reset_index()

,user_id,price_fixed
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


In [121]:
columns = ['user_id', 'recommend']
user_revs = pd.read_csv('./datasets/aus_user_revs_clean.csv', usecols=columns)



In [122]:
user_revs.sample(5)

,user_id,recommend
6602,76561198140052687,True
4990,Revolver_,True
31248,76561198157987157,True
58257,76561198115910259,True
51946,Katzieon,False


In [123]:
user_gb_count_revs = pd.DataFrame()

user_gb_count_revs['count_total'] = user_revs.groupby('user_id')['recommend'].agg('count')
user_gb_count_revs['count_true'] = user_revs[user_revs['recommend'] == True].groupby('user_id')['recommend'].agg(count_revs='count')


user_gb_count_revs.reset_index(inplace=True)
user_gb_count_revs

# proporción
user_gb_count_revs['reco_porcentaje'] = user_gb_count_revs['count_true'] / user_gb_count_revs['count_total']
user_gb_count_revs.fillna(0,inplace=True)

In [124]:
user_gb_count_revs.sample(5)

,user_id,count_total,count_true,reco_porcentaje
22561,mylordbendtner,3,2.0,0.666667
7104,76561198072813477,1,1.0,1.000000
17043,SuddenNight,1,1.0,1.000000
24845,umicornbl,1,1.0,1.000000
371,45514551,4,4.0,1.000000


In [125]:
user_revs[user_revs['user_id']=='iliakitmail']

,user_id,recommend
27618,iliakitmail,False


In [127]:
df_merge2 = user_gb_count_revs.merge(user_spent, on='user_id',how='left' )
df_merge2.rename(columns={'price_fixed':'total_spent'},inplace=True)


In [136]:
df_merge3 = df_merge2.merge(user_items[['user_id','items_count']].drop_duplicates(),on='user_id',how='left')
df_merge3 = df_merge3.drop(['count_total','count_true'],axis=1)
df_merge3['reco_porcentaje'] = df_merge3['reco_porcentaje'].apply(lambda x : int(x *100))
df_merge3.rename(columns={'reco_porcentaje':'porcentaje_recomendacion', 'total_spent':'cantidad_gastada', 'items_count':'cantidad_items'},inplace=True)

In [137]:
df_merge3.to_csv(r'dataquery/user_data.csv',index=False)

In [138]:
def userdata(user_id : str):
    df_user = pd.read_csv('dataquery/user_data.csv')
    user_data = df_user[df_user['user_id'] == user_id]
    
    return user_data.to_json(orient='records')

In [139]:
userdata('--000--')

'[{"user_id":"--000--","procentaje_recomendacion":100,"cantidad_gastada":402.77,"cantidad_items":58.0}]'

### Consulta 6
def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

- Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

- archivo aus_user_clean, columna sentiment analysis: 0 negativo, 1 neutro, 2 positivo

In [33]:
import pandas as pd

columns = ['user_id','item_id', 'sentiment_analysis']
user_revs = pd.read_csv('datasets/aus_user_revs_clean.csv', usecols=columns)
user_revs.head()

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2.0
1,76561197970982479,22200,2.0
2,76561197970982479,43110,2.0
3,js41637,251610,2.0
4,js41637,227300,2.0


In [34]:
user_revs['item_id'] = user_revs['item_id'].astype(float)

In [35]:
user_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59305 non-null  object 
 1   item_id             59305 non-null  float64
 2   sentiment_analysis  59275 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.4+ MB


In [36]:
user_revs.isna().sum()

user_id                0
item_id                0
sentiment_analysis    30
dtype: int64

In [37]:
user_revs.dropna(inplace=True)

In [38]:
columns = ['id', 'release_date']
games = pd.read_csv('datasets/steam_games.csv', usecols=columns).rename(columns={'id':'item_id'})
games.sample(15)

,release_date,item_id
4283,2015-08-27,325962.0
10520,2013-06-25,669940.0
7615,2016-10-18,435000.0
25619,2015-09-09,392880.0
24878,2015-11-20,381191.0
25509,2018-02-28,406090.0
4833,2014-09-01,321060.0
9457,2005-01-01,594870.0
4042,2015-07-21,369783.0
28709,2014-07-10,279820.0


In [39]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  30068 non-null  object 
 1   item_id       32133 non-null  float64
dtypes: float64(1), object(1)
memory usage: 502.2+ KB


In [40]:
games.isna().sum()

release_date    2067
item_id            2
dtype: int64

In [17]:
games.dropna(inplace=True)

In [19]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30067 entries, 0 to 32133
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  30067 non-null  object 
 1   item_id       30067 non-null  float64
dtypes: float64(1), object(1)
memory usage: 704.7+ KB


In [82]:
df_sa = user_revs.merge(games, on='item_id', how='left')
df_sa.sample(10)

,user_id,item_id,sentiment_analysis,release_date
55917,7656119815549OO43,227940.0,2.0,2016-10-18
19489,spyron,440.0,1.0,2007-10-10
11457,76561198062533699,301520.0,2.0,2017-08-24
408,76561198061174192,220200.0,2.0,2015-04-27
36825,Nz1_,236390.0,2.0,NaN
29379,certifiedweabooz,212680.0,0.0,2012-09-14
28125,Dazie,319630.0,2.0,2015-01-29
33808,76561198013785384,105600.0,1.0,2011-05-16
10013,chewyi,209650.0,1.0,2014-11-03
20285,megaglitch016,440.0,0.0,2007-10-10


In [83]:
df_sa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59275 entries, 0 to 59274
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59275 non-null  object 
 1   item_id             59275 non-null  float64
 2   sentiment_analysis  59275 non-null  float64
 3   release_date        50555 non-null  object 
dtypes: float64(2), object(2)
memory usage: 1.8+ MB


In [84]:
df_sa.isna().sum()

user_id                  0
item_id                  0
sentiment_analysis       0
release_date          8720
dtype: int64

In [85]:
#df_sa.dropna(inplace=True) # o con la moda?
df_sa.fillna(df_sa['release_date'].mode()[0], inplace=True)

In [ ]:
import pandas as pd
import re

def safe_date_convert(df):
    def convert(x):
        try:
            return pd.to_datetime(x)
        except ValueError:
            # Try to find a year pattern in the string
            year_pattern = re.search(r'\b\d{4}\b', x)
            if year_pattern:
                # Convert the found year into a datetime format
                return pd.to_datetime(year_pattern.group(0), format='%Y')
            else:
                # If no year pattern is found, return NaT
                return pd.NaT

    df['release_date_fixed'] = df['release_date'].astype(str).apply(convert)
    return df

In [87]:
df_sa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59275 entries, 0 to 59274
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             59275 non-null  object        
 1   item_id             59275 non-null  float64       
 2   sentiment_analysis  59275 non-null  float64       
 3   release_date        59275 non-null  object        
 4   converted_date      59275 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 2.3+ MB


In [89]:
df_sa['year_released'] = df_sa['converted_date'].dt.year
df_sa.head()

,user_id,item_id,sentiment_analysis,release_date,converted_date,year_released
0,76561197970982479,1250.0,2.0,2009-05-14,2009-05-14,2009
1,76561197970982479,22200.0,2.0,2009-04-21,2009-04-21,2009
2,76561197970982479,43110.0,2.0,2007-10-10,2007-10-10,2007
3,js41637,251610.0,2.0,2007-10-10,2007-10-10,2007
4,js41637,227300.0,2.0,2013-01-16,2013-01-16,2013


In [92]:
df_sa = df_sa[['sentiment_analysis','year_released']]
df_sa.head()

,sentiment_analysis,year_released
0,2.0,2009
1,2.0,2009
2,2.0,2007
3,2.0,2007
4,2.0,2013


In [102]:
sentiment_analysis = pd.DataFrame()
sentiment_analysis['Negative'] = df_sa[df_sa['sentiment_analysis']==0].groupby('year_released').agg('count')
sentiment_analysis['Neutral'] = df_sa[df_sa['sentiment_analysis']==1].groupby('year_released').agg('count')
sentiment_analysis['Positive'] = df_sa[df_sa['sentiment_analysis']==2].groupby('year_released').agg('count')
sentiment_analysis.fillna(0, inplace=True)
sentiment_analysis.sample(10)

,Negative,Neutral,Positive
year_released,,,
1993,3,0.0,2
2001,8,2.0,25
2007,1590,3271.0,8031
1990,1,0.0,4
2014,1023,1332.0,4091
2005,37,75.0,182
2003,42,63.0,112
2000,8,33.0,58
1997,8,9.0,31


In [103]:
sentiment_analysis.to_csv('dataquery/sentiment_analysis.csv')

In [105]:
df = pd.read_csv('dataquery/sentiment_analysis.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year_released  26 non-null     int64  
 1   Negative       26 non-null     int64  
 2   Neutral        26 non-null     float64
 3   Positive       26 non-null     int64  
dtypes: float64(1), int64(3)
memory usage: 960.0 bytes


In [119]:
def sentiment_analysis(año : str):
    df = pd.read_csv('dataquery/sentiment_analysis.csv')
    df['year_released'] = df['year_released'].astype(str)
    if df['year_released'].str.contains(año).any():
        return df[df['year_released'] == año].to_json(orient='records')
    else:
        return 'Year not found'
        

In [120]:
sentiment_analysis('2014')

'[{"year_released":"2014","Negative":1023,"Neutral":1332.0,"Positive":4091}]'

### Consulta 5
def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [254]:
columns = ['id', 'price', 'developer','title', 'release_date']
games = pd.read_csv('datasets/steam_games.csv', usecols=columns).rename(columns={'id':'item_id'})
games.sample(10)

,title,release_date,price,item_id,developer
21524,Albino Lullaby Demo,2016-09-14,NaN,367320.0,Ape Law
3745,Flame Over,2015-05-28,11.99,345080.0,Laughing Jackal LTD
7436,Zombie Defense,2016-09-30,Free To Play,533780.0,Home Net Games
16966,NaN,NaN,4.99,601110.0,NaN
29964,Knock-knock,2013-10-04,9.99,250380.0,Ice-Pick Lodge
17449,Hacktag,2017-06-01,14.99,622770.0,Piece of Cake studios
8503,Hollow Halls,2017-01-19,2.99,576700.0,Broken Code Games
7499,SparkDimension,2016-10-04,9.99,477140.0,Asixa
12030,EVERSPACE™ - Encounters,2017-10-24,9.99,709150.0,ROCKFISH Games
26801,Rustbucket Rumble Soundtrack,2015-04-02,1.99,356780.0,Reactor Zero


In [255]:
games = fix_price(games)
games.head()

,title,release_date,price,item_id,developer,price_fixed
0,Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro,4.99
1,Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL,0
2,Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com,0
3,弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域,0.99
4,NaN,NaN,2.99,773570.0,NaN,2.99


In [249]:
import pandas as pd
import re

def safe_date_convert(df):
    def convert(x):
        try:
            return pd.to_datetime(x)
        except ValueError:
            # Try to find a year pattern in the string
            year_pattern = re.search(r'\b\d{4}\b', x)
            if year_pattern:
                # Convert the found year into a datetime format
                return pd.to_datetime(year_pattern.group(0), format='%Y')
            else:
                # If no year pattern is found, return NaT
                return pd.NaT

    df['release_date_fixed'] = df['release_date'].astype(str).apply(convert)
    return df


In [256]:
games = safe_date_convert(games)
games.head()

/var/folders/g_/fkn0ttqn6w5_k96kvk3ctwm80000gn/T/ipykernel_26579/3336880464.py:7: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(x)


,title,release_date,price,item_id,developer,price_fixed,release_date_fixed
0,Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro,4.99,2018-01-04
1,Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL,0,2018-01-04
2,Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com,0,2017-07-24
3,弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域,0.99,2017-12-07
4,NaN,NaN,2.99,773570.0,NaN,2.99,NaT


In [257]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               30085 non-null  object        
 1   release_date        30068 non-null  object        
 2   price               30758 non-null  object        
 3   item_id             32133 non-null  float64       
 4   developer           28836 non-null  object        
 5   price_fixed         30758 non-null  object        
 6   release_date_fixed  29964 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.7+ MB


In [258]:
games.isna().sum()

title                 2050
release_date          2067
price                 1377
item_id                  2
developer             3299
price_fixed           1377
release_date_fixed    2171
dtype: int64

In [259]:
games['release_year'] = games['release_date_fixed'].dt.year
games.head()

,title,release_date,price,item_id,developer,price_fixed,release_date_fixed,release_year
0,Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro,4.99,2018-01-04,2018.0
1,Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL,0,2018-01-04,2018.0
2,Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com,0,2017-07-24,2017.0
3,弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域,0.99,2017-12-07,2017.0
4,NaN,NaN,2.99,773570.0,NaN,2.99,NaT,NaN


In [263]:
games.drop(columns=['release_date','price','release_date_fixed'],inplace=True)
games.drop(columns=['title'], inplace=True)
games.head()

,item_id,developer,price_fixed,release_year
0,761140.0,Kotoshiro,4.99,2018.0
1,643980.0,Secret Level SRL,0,2018.0
2,670290.0,Poolians.com,0,2017.0
3,767400.0,彼岸领域,0.99,2017.0
4,773570.0,NaN,2.99,NaN


In [264]:
games.isna().sum()

item_id            2
developer       3299
price_fixed     1377
release_year    2171
dtype: int64

In [265]:
# imputo precio como 0 a los na y a release year por la moda
# Los nulos de desarrollador no puedo imputarlos, así que los elimino.

games.dropna(subset=['developer'], inplace=True)


In [266]:
games['price_fixed'].fillna(0,inplace=True)
games['release_year'].fillna(games['release_year'].mode().iloc[0],inplace=True)

In [267]:
games.isna().sum()

item_id         1
developer       0
price_fixed     0
release_year    0
dtype: int64

In [268]:
games.duplicated().sum()

1

In [269]:
games.drop_duplicates(inplace=True)

In [270]:
# Primera aggregation
dev = games.groupby(['developer', 'release_year'], as_index=False).agg({'item_id':'count'}).rename(columns={'item_id':'item_count'})

# Segunda aggregation
free_games = games[games['price_fixed']==0].groupby(['developer','release_year'], as_index=False)['item_id'].agg('count').rename(columns={'item_id':'free'})

# Merge
dev = pd.merge(dev, free_games, on=['developer', 'release_year'], how='left')

# Imputar nans
dev['free'].fillna(0,inplace=True)

# Calcular porcentaje
dev['porcentaje_free'] = dev['free'] / dev['item_count'] * 100

# Sample
dev.sample(10)


,developer,release_year,item_count,free,porcentaje_free
9637,PixelGreeds,2017.0,1,0.0,0.0
5304,Gestmorph Games,2016.0,1,1.0,100.0
9529,Phosphor Games Studio,2014.0,1,0.0,0.0
6027,ILIKESCIFI Games,2014.0,2,1.0,50.0
11747,"Spike Chunsoft Co., Ltd.",2017.0,2,0.0,0.0
7870,"Massive Damage, Inc.",2016.0,2,1.0,50.0
4483,Far Mills,2017.0,1,0.0,0.0
12283,TEN TIULENYA team,2016.0,1,0.0,0.0
11361,Sinister Design,2015.0,1,0.0,0.0
3758,Dragon Wing Studio,2017.0,6,0.0,0.0


In [273]:
dev['release_year'] = dev['release_year'].astype(int)
dev['developer'] = dev['developer'].str.strip().str.lower()
dev['porcentaje_free'] = dev['porcentaje_free'].round(2)
dev.drop(columns=['free'], inplace=True)
dev.sample(10)

,developer,release_year,item_count,porcentaje_free
11304,silver cow studio,2015,1,0.00
9136,oovee® game studios,2016,1,0.00
12922,tomatotrap,2017,1,0.00
12513,techland,2007,1,0.00
6143,illwinter game design,2012,1,0.00
11675,spark unlimited,2013,6,0.00
13813,warped tales,2017,3,33.33
10076,rex,2016,1,0.00
8454,mystik'art,2015,1,0.00
12816,thunder lotus games,2017,2,0.00


In [283]:
dev.sort_values(by='item_count', ascending=False)

,developer,release_year,item_count,porcentaje_free
11507,"smiteworks usa, llc",2017,428,1.17
13258,ubisoft - san francisco,2017,269,0.00
13255,ubisoft - san francisco,2014,265,0.00
13256,ubisoft - san francisco,2015,248,0.00
13257,ubisoft - san francisco,2016,247,0.00
...,...,...,...,...
5847,hip soft,2008,1,0.00
5849,hipix studio,2015,1,0.00
5851,hipix studio,2017,1,0.00
5853,hit the crow,2015,1,0.00


In [274]:
dev.to_csv('dataquery/developer.csv',index=False)

In [277]:
developer = 'Bigosaur'
data = dev[dev['developer'] == developer.strip().lower()]
data = data[['release_year', 'item_count', 'porcentaje_free']]
data

,developer,release_year,item_count,porcentaje_free
1539,bigosaur,2015,1,0.0
1540,bigosaur,2016,1,0.0


In [279]:
def developer(desarrollador : str):
    df = pd.read_csv('dataquery/developer.csv')
    desarrollador = desarrollador.strip().lower()
    if df['developer'].str.contains(desarrollador).any():
        data = df[df['developer'] == desarrollador]
        response = data[['release_year','item_count','porcentaje_free']].to_dict(orient='records')
        return response
    else:
        return 'Error: Developer not found'

In [284]:
developer('ubisoft - san francisco')

[{'release_year': 2012, 'item_count': 114, 'porcentaje_free': 0.88},
 {'release_year': 2013, 'item_count': 116, 'porcentaje_free': 0.0},
 {'release_year': 2014, 'item_count': 265, 'porcentaje_free': 0.0},
 {'release_year': 2015, 'item_count': 248, 'porcentaje_free': 0.0},
 {'release_year': 2016, 'item_count': 247, 'porcentaje_free': 0.0},
 {'release_year': 2017, 'item_count': 269, 'porcentaje_free': 0.0}]